<a href="https://colab.research.google.com/github/overred0704/Stock_filter/blob/main/stock_filter_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#gogo

##import package

In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import time

##set crawler

In [ ]:
def crawl_price(date):
    url = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    df = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in url.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    df = df.set_index('證券代號')
    df['成交金額'] = df['成交金額'].str.replace(',','')
    df['成交股數'] = df['成交股數'].str.replace(',','')
    return df

##set runing

In [ ]:
def running(n_days=3, max_fail_count = 4, date=datetime.datetime.now()):
  
  #variable initial
  data = {}
  fail_count = 0

  while n_days > 0 :

    print('loading', date)
    # crawler
    try:
        # start
        data[date.date()] = crawl_price(date)
        print('complete!')
        fail_count = 0
    except:
        # 跳過假日
        print('fail!')
        fail_count += 1
    else:
      n_days -= 1
    if fail_count > max_fail_count:
      break
    
    #update day
    date -= datetime.timedelta(days=1)
    time.sleep(20)

    

  #取得股數
  df = pd.DataFrame({k:d['成交股數'] for k,d in data.items()}).transpose()
  df.index = pd.to_datetime(df.index)
  c = df.transpose()   

  #剔除空值
  c.dropna(axis=0, inplace=True)

  #convert data type and rename columns
  c = c.astype(str).astype(int)
  c.columns = ['今天', '昨天','前天']

  return c

#set

In [ ]:
da = datetime.datetime(2021,2,4)
print(da)

In [ ]:
df = running(date=da)

loading 2021-02-04 00:00:00
fail!
loading 2021-02-03 00:00:00
complete!
loading 2021-02-02 00:00:00
complete!
loading 2021-02-01 00:00:00
complete!


In [ ]:
w1 = (1/4)
w2 = 3

In [ ]:
#calculate threshold
df['threshold1'] = df['今天'] - w1 * df['昨天']
df['threshold2'] = df['前天'] * w2 - df['昨天']

In [ ]:
#set filter
f1 = df['threshold1'] < 0
f2 = df['threshold2'] < 0

In [ ]:
#write to new df
df_new = df[(f1 & f2 )].sort_values('前天',ascending=False)

df_new = df_new.drop(['threshold1', 'threshold2'], axis=1)

df_new.index.name = None

In [ ]:
df_new

,今天,昨天,前天
2491,897340,7242205,755611
2365,982889,11376957,447650
1416,98672,770842,187393
2509,159150,683934,95539
2849,74280,396393,59584
2891B,1306,262632,24712
1453,4463,94246,10696
1413,24123,117969,6000
2897A,0,23000,5000
5906,2025,14464,3100


#filtering function

In [ ]:
def filtering(w1 = (1/3), w2 = 3):
  df = running()

  #calculate threshold
  df['threshold1'] = df['今天'] - w1 * df['昨天']
  df['threshold2'] = df['前天'] * w2 - df['昨天']

  #set filter
  f1 = df['threshold1'] < 0
  f2 = df['threshold2'] < 0

  #write to new df
  df_new = df[(f1 & f2 )].sort_values('前天',ascending=False)

  df_new = df_new.drop(['threshold1', 'threshold2'], axis=1)

  df_new.index.name = None

  return df_new

In [ ]:
test = filtering()

loading 2021-06-18 09:10:49.389117
complete!
loading 2021-06-17 09:10:49.389117
complete!
loading 2021-06-16 09:10:49.389117
complete!
